In [1]:
#we have the hyena utils, let's see if we can quickly adapt it for caduceus

import sys
sys.path.append('/data/leslie/sarthak/caduceus')
from src.models.sequence.dna_embedding import DNAEmbeddingModel
from src.tasks.decoders import EnformerDecoder
# from src.tasks.encoders import EnformerEncoder
import torch
import numpy as np
import src.dataloaders.datasets.enformer_dataset as enformer_dataset
import yaml
from omegaconf import OmegaConf
import os
import matplotlib.pyplot as plt
# import seaborn as sns
from tqdm import tqdm

In [2]:
try:
    OmegaConf.register_new_resolver('eval', eval)
    OmegaConf.register_new_resolver('div_up', lambda x, y: (x + y - 1) // y)
except ValueError as e:
    if "Resolver already registered" in str(e):
            print(f"Resolver already exists, skipping registration.")

In [3]:
class IdentityNet(torch.nn.Module):
    def __init__(self):
        super(IdentityNet, self).__init__()

    def forward(self, x):
        return x, None

In [3]:
ckpt_path = '/data/leslie/sarthak/caduceus/outputs/2024-08-06/16-29-46-520749/checkpoints/05-val_loss=0.61190.ckpt'
model_cfg_path = os.path.join(os.path.dirname(os.path.dirname(ckpt_path)), '.hydra', 'config.yaml')

In [4]:
cfg = yaml.load(open(model_cfg_path, 'r'), Loader=yaml.FullLoader)
cfg = OmegaConf.create(cfg)
cfg = OmegaConf.to_container(cfg, resolve=True)
state_dict = torch.load(ckpt_path, map_location='cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
cfg #lets you see the evaluated config and things like accumulate grad batches is 1

{'train': {'seed': 2222,
  'interval': 'epoch',
  'monitor': 'val/loss',
  'mode': 'min',
  'ema': 0.0,
  'test': True,
  'debug': False,
  'ignore_warnings': False,
  'optimizer_param_grouping': {'bias_weight_decay': False,
   'normalization_weight_decay': False},
  'state': {'mode': None, 'n_context': 0, 'n_context_eval': 0},
  'ckpt': 'checkpoints/last.ckpt',
  'disable_dataset': False,
  'validate_at_start': False,
  'pretrained_model_path': None,
  'pretrained_model_strict_load': False,
  'pretrained_model_state_hook': {'_name_': 'load_backbone',
   'freeze_backbone': False,
   'ignore_head': True,
   'add_embeddings': False,
   'ignore_embeddings': False},
  'post_init_hook': {'_name_': None},
  'layer_decay': {'_name_': None, 'decay': 0.7},
  'gpu_mem': 25,
  'global_batch_size': 1,
  'pretrained_safetensors_model_path': None},
 'wandb': {'project': 'dna',
  'group': 'enformer',
  'job_type': 'training',
  'mode': 'online',
  'name': 'enformer_kmer_mamba_1',
  'save_dir': '.',
 

In [6]:
dataset_args = cfg['dataset']

In [8]:
split = 'test'
dataset = enformer_dataset.EnformerDataset(split, dataset_args['max_length'], rc_aug = dataset_args['rc_aug'],
                                                            return_CAGE=dataset_args['return_cage'], cell_type=dataset_args.get('cell_type', None),
                                                            kmer_len=dataset_args['kmer_len']) #could use dataloader instead, but again kinda complex

Using kmer genome with length 6


In [16]:
dataset.complement_map

AttributeError: 'EnformerDataset' object has no attribute 'complement_map'

In [10]:
cfg['decoder']['d_output'] = dataset.d_output

In [17]:
cfg['model']['config']['complement_map'] = dataset.tokenizer.complement_map

In [11]:
torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
    state_dict["state_dict"], "model."
)
model_state_dict = state_dict["state_dict"]
# need to remove torchmetrics. to remove keys, need to convert to list first
for key in list(model_state_dict.keys()):
    if "torchmetrics" in key:
        model_state_dict.pop(key)
# the state_dict keys slightly mismatch from Lightning..., so we fix it here
decoder_state_dict = {}
for key in list(model_state_dict.keys()):
    if "decoder" in key:
        decoder_state_dict[key[10:]] = model_state_dict.pop(key)
encoder_state_dict = {}
for key in list(model_state_dict.keys()):
    if "encoder" in key:
        encoder_state_dict[key[10:]] = model_state_dict.pop(key)

In [19]:
cfg['model']['config'].keys()

dict_keys(['_target_', 'd_model', 'n_layer', 'vocab_size', 'ssm_cfg', 'rms_norm', 'fused_add_norm', 'residual_in_fp32', 'pad_vocab_size_multiple', 'norm_epsilon', 'initializer_cfg', 'bidirectional', 'bidirectional_strategy', 'bidirectional_weight_tie', 'rcps', 'complement_map'])

In [20]:
import copy
tempcfg = copy.copy(cfg['model']['config'])
tempcfg.pop('complement_map')

{'0': 11718,
 '1': 8593,
 '2': 5468,
 '3': 2343,
 '4': 14843,
 '5': 11093,
 '6': 7968,
 '7': 4843,
 '8': 1718,
 '9': 14218,
 '10': 10468,
 '11': 7343,
 '12': 4218,
 '13': 1093,
 '14': 13593,
 '15': 9843,
 '16': 6718,
 '17': 3593,
 '18': 468,
 '19': 12968,
 '20': 12343,
 '21': 9218,
 '22': 6093,
 '23': 2968,
 '24': 15468,
 '25': 11593,
 '26': 8468,
 '27': 5343,
 '28': 2218,
 '29': 14718,
 '30': 10968,
 '31': 7843,
 '32': 4718,
 '33': 1593,
 '34': 14093,
 '35': 10343,
 '36': 7218,
 '37': 4093,
 '38': 968,
 '39': 13468,
 '40': 9718,
 '41': 6593,
 '42': 3468,
 '43': 343,
 '44': 12843,
 '45': 12218,
 '46': 9093,
 '47': 5968,
 '48': 2843,
 '49': 15343,
 '50': 11468,
 '51': 8343,
 '52': 5218,
 '53': 2093,
 '54': 14593,
 '55': 10843,
 '56': 7718,
 '57': 4593,
 '58': 1468,
 '59': 13968,
 '60': 10218,
 '61': 7093,
 '62': 3968,
 '63': 843,
 '64': 13343,
 '65': 9593,
 '66': 6468,
 '67': 3343,
 '68': 218,
 '69': 12718,
 '70': 12093,
 '71': 8968,
 '72': 5843,
 '73': 2718,
 '74': 15218,
 '75': 11343,

In [21]:
tempcfg

{'_target_': 'caduceus.configuration_caduceus.CaduceusConfig',
 'd_model': 256,
 'n_layer': 16,
 'vocab_size': 15625,
 'ssm_cfg': {'d_state': 16,
  'd_conv': 4,
  'expand': 2,
  'dt_rank': 'auto',
  'dt_min': 0.001,
  'dt_max': 0.1,
  'dt_init': 'random',
  'dt_scale': 1.0,
  'dt_init_floor': 0.0001,
  'conv_bias': True,
  'bias': False,
  'use_fast_path': True},
 'rms_norm': True,
 'fused_add_norm': True,
 'residual_in_fp32': False,
 'pad_vocab_size_multiple': 1,
 'norm_epsilon': 1e-05,
 'initializer_cfg': {'initializer_range': 0.02,
  'rescale_prenorm_residual': True,
  'n_residuals_per_layer': 1},
 'bidirectional': True,
 'bidirectional_strategy': 'add',
 'bidirectional_weight_tie': True,
 'rcps': True}

In [30]:
tempcfg = copy.copy(cfg['model']['config'])
tempcfg.pop('_target_')

'caduceus.configuration_caduceus.CaduceusConfig'

In [25]:
backbone = DNAEmbeddingModel(**tempcfg)

TypeError: DNAEmbeddingModel.__init__() missing 1 required positional argument: 'd_inner'

In [32]:
tempcfg.pop('d_model')

256

In [33]:
from src.models.sequence.dna_embedding import DNAEmbeddingModelCaduceus
backbone = DNAEmbeddingModelCaduceus(**tempcfg)

TypeError: DNAEmbeddingModelCaduceus.__init__() got an unexpected keyword argument 'n_layer'

In [34]:
#wait that's all messed up, we need to put the config as the input
backbone = DNAEmbeddingModelCaduceus(**cfg['model'])

TypeError: DNAEmbeddingModelCaduceus.__init__() got an unexpected keyword argument '_name_'

In [37]:
temp_cfg = copy.copy(cfg['model'])
temp_cfg['config'].pop('_target_')

'caduceus.configuration_caduceus.CaduceusConfig'

In [38]:
temp_cfg.pop('_name_')

'dna_embedding_caduceus'

In [39]:
backbone = DNAEmbeddingModelCaduceus(**temp_cfg)

AttributeError: 'dict' object has no attribute 'd_model'

In [41]:
temp_cfg

{'config': {'d_model': 256,
  'n_layer': 16,
  'vocab_size': 15625,
  'ssm_cfg': {'d_state': 16,
   'd_conv': 4,
   'expand': 2,
   'dt_rank': 'auto',
   'dt_min': 0.001,
   'dt_max': 0.1,
   'dt_init': 'random',
   'dt_scale': 1.0,
   'dt_init_floor': 0.0001,
   'conv_bias': True,
   'bias': False,
   'use_fast_path': True},
  'rms_norm': True,
  'fused_add_norm': True,
  'residual_in_fp32': False,
  'pad_vocab_size_multiple': 1,
  'norm_epsilon': 1e-05,
  'initializer_cfg': {'initializer_range': 0.02,
   'rescale_prenorm_residual': True,
   'n_residuals_per_layer': 1},
  'bidirectional': True,
  'bidirectional_strategy': 'add',
  'bidirectional_weight_tie': True,
  'rcps': True,
  'complement_map': {'0': 11718,
   '1': 8593,
   '2': 5468,
   '3': 2343,
   '4': 14843,
   '5': 11093,
   '6': 7968,
   '7': 4843,
   '8': 1718,
   '9': 14218,
   '10': 10468,
   '11': 7343,
   '12': 4218,
   '13': 1093,
   '14': 13593,
   '15': 9843,
   '16': 6718,
   '17': 3593,
   '18': 468,
   '19': 129

In [42]:
#perhaps the answer is to set all the dict options as attributes...
class Options:
    def __init__(self, options_dict):
        for key, value in options_dict.items():
            setattr(self, key, value)
            
finalcfg = Options(cfg['model']['config'])

In [44]:
backbone = DNAEmbeddingModelCaduceus(config=finalcfg)

ValueError: Parameter config in `Caduceus(config)` should be an instance of class `PretrainedConfig`. To create a model from a pretrained model use `model = Caduceus.from_pretrained(PRETRAINED_MODEL_NAME)`

In [46]:
#we need to find how caduceus creates it's config...
from caduceus.configuration_caduceus import CaduceusConfig
caduceus_cfg = CaduceusConfig(cfg['model']['config'])

In [47]:
caduceus_cfg

CaduceusConfig {
  "bidirectional": true,
  "bidirectional_strategy": "add",
  "bidirectional_weight_tie": true,
  "complement_map": {
    "0": 0,
    "1": 1,
    "10": 7,
    "11": 11,
    "12": 12,
    "13": 13,
    "14": 14,
    "15": 15,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 10,
    "8": 9,
    "9": 8
  },
  "d_model": {
    "bidirectional": true,
    "bidirectional_strategy": "add",
    "bidirectional_weight_tie": true,
    "complement_map": {
      "0": 11718,
      "1": 8593,
      "10": 10468,
      "100": 11843,
      "1000": 11638,
      "10000": 11710,
      "10001": 8585,
      "10002": 5460,
      "10003": 2335,
      "10004": 14835,
      "10005": 11085,
      "10006": 7960,
      "10007": 4835,
      "10008": 1710,
      "10009": 14210,
      "1001": 8513,
      "10010": 10460,
      "10011": 7335,
      "10012": 4210,
      "10013": 1085,
      "10014": 13585,
      "10015": 9835,
      "10016": 6710,
      "10017": 3585,
      "10018": 46

In [48]:
#wrong complement map so let's fix it!
caduceus_cfg.complement_map = dataset.tokenizer.complement_map

In [49]:
caduceus_cfg

CaduceusConfig {
  "bidirectional": true,
  "bidirectional_strategy": "add",
  "bidirectional_weight_tie": true,
  "complement_map": {
    "0": 11718,
    "1": 8593,
    "10": 10468,
    "100": 11843,
    "1000": 11638,
    "10000": 11710,
    "10001": 8585,
    "10002": 5460,
    "10003": 2335,
    "10004": 14835,
    "10005": 11085,
    "10006": 7960,
    "10007": 4835,
    "10008": 1710,
    "10009": 14210,
    "1001": 8513,
    "10010": 10460,
    "10011": 7335,
    "10012": 4210,
    "10013": 1085,
    "10014": 13585,
    "10015": 9835,
    "10016": 6710,
    "10017": 3585,
    "10018": 460,
    "10019": 12960,
    "1002": 5388,
    "10020": 12335,
    "10021": 9210,
    "10022": 6085,
    "10023": 2960,
    "10024": 15460,
    "10025": 11585,
    "10026": 8460,
    "10027": 5335,
    "10028": 2210,
    "10029": 14710,
    "1003": 2263,
    "10030": 10960,
    "10031": 7835,
    "10032": 4710,
    "10033": 1585,
    "10034": 14085,
    "10035": 10335,
    "10036": 7210,
    "10037

In [50]:
#now let's try the model
backbone = DNAEmbeddingModelCaduceus(config=caduceus_cfg)

TypeError: empty(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got dict"

In [52]:
caduceus_cfg.vocab_size

50280

In [53]:
caduceus_cfg.d_model

{'d_model': 256,
 'n_layer': 16,
 'vocab_size': 15625,
 'ssm_cfg': {'d_state': 16,
  'd_conv': 4,
  'expand': 2,
  'dt_rank': 'auto',
  'dt_min': 0.001,
  'dt_max': 0.1,
  'dt_init': 'random',
  'dt_scale': 1.0,
  'dt_init_floor': 0.0001,
  'conv_bias': True,
  'bias': False,
  'use_fast_path': True},
 'rms_norm': True,
 'fused_add_norm': True,
 'residual_in_fp32': False,
 'pad_vocab_size_multiple': 1,
 'norm_epsilon': 1e-05,
 'initializer_cfg': {'initializer_range': 0.02,
  'rescale_prenorm_residual': True,
  'n_residuals_per_layer': 1},
 'bidirectional': True,
 'bidirectional_strategy': 'add',
 'bidirectional_weight_tie': True,
 'rcps': True,
 'complement_map': {'0': 11718,
  '1': 8593,
  '2': 5468,
  '3': 2343,
  '4': 14843,
  '5': 11093,
  '6': 7968,
  '7': 4843,
  '8': 1718,
  '9': 14218,
  '10': 10468,
  '11': 7343,
  '12': 4218,
  '13': 1093,
  '14': 13593,
  '15': 9843,
  '16': 6718,
  '17': 3593,
  '18': 468,
  '19': 12968,
  '20': 12343,
  '21': 9218,
  '22': 6093,
  '23': 29

# let's just get the config from scratch

In [1]:
#we have the hyena utils, let's see if we can quickly adapt it for caduceus

import sys
sys.path.append('/data/leslie/sarthak/caduceus')
from src.models.sequence.dna_embedding import DNAEmbeddingModel
from src.tasks.decoders import EnformerDecoder
# from src.tasks.encoders import EnformerEncoder
import torch
import numpy as np
import src.dataloaders.datasets.enformer_dataset as enformer_dataset
import yaml
from omegaconf import OmegaConf
import os
import matplotlib.pyplot as plt
# import seaborn as sns
from tqdm import tqdm
try:
    OmegaConf.register_new_resolver('eval', eval)
    OmegaConf.register_new_resolver('div_up', lambda x, y: (x + y - 1) // y)
except ValueError as e:
    if "Resolver already registered" in str(e):
            print(f"Resolver already exists, skipping registration.")
class IdentityNet(torch.nn.Module):
    def __init__(self):
        super(IdentityNet, self).__init__()

    def forward(self, x):
        return x, None
ckpt_path = '/data/leslie/sarthak/caduceus/outputs/2024-08-06/16-29-46-520749/checkpoints/05-val_loss=0.61190.ckpt'
model_cfg_path = os.path.join(os.path.dirname(os.path.dirname(ckpt_path)), '.hydra', 'config.yaml')
cfg = yaml.load(open(model_cfg_path, 'r'), Loader=yaml.FullLoader)
cfg = OmegaConf.create(cfg)
cfg = OmegaConf.to_container(cfg, resolve=True)
state_dict = torch.load(ckpt_path, map_location='cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_args = cfg['dataset']
split = 'test'
dataset = enformer_dataset.EnformerDataset(split, dataset_args['max_length'], rc_aug = dataset_args['rc_aug'],
                                                            return_CAGE=dataset_args['return_cage'], cell_type=dataset_args.get('cell_type', None),
                                                            kmer_len=dataset_args['kmer_len']) #could use dataloader instead, but again kinda complex

Using kmer genome with length 6


In [2]:
cfg['model']['config']['complement_map'] = dataset.tokenizer.complement_map

In [12]:
cfg['model']['config'].keys()

dict_keys(['_target_', 'd_model', 'n_layer', 'vocab_size', 'ssm_cfg', 'rms_norm', 'fused_add_norm', 'residual_in_fp32', 'pad_vocab_size_multiple', 'norm_epsilon', 'initializer_cfg', 'bidirectional', 'bidirectional_strategy', 'bidirectional_weight_tie', 'rcps', 'complement_map'])

In [15]:
cfg['model']['config'].pop('_target_')

'caduceus.configuration_caduceus.CaduceusConfig'

In [18]:
cfg['model']['config'].keys()

dict_keys(['d_model', 'n_layer', 'vocab_size', 'ssm_cfg', 'rms_norm', 'fused_add_norm', 'residual_in_fp32', 'pad_vocab_size_multiple', 'norm_epsilon', 'initializer_cfg', 'bidirectional', 'bidirectional_strategy', 'bidirectional_weight_tie', 'rcps', 'complement_map'])

In [16]:
from caduceus.configuration_caduceus import CaduceusConfig
caduceus_cfg = CaduceusConfig(cfg['model']['config'])

In [17]:
print(caduceus_cfg)

CaduceusConfig {
  "bidirectional": true,
  "bidirectional_strategy": "add",
  "bidirectional_weight_tie": true,
  "complement_map": {
    "0": 0,
    "1": 1,
    "10": 7,
    "11": 11,
    "12": 12,
    "13": 13,
    "14": 14,
    "15": 15,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 10,
    "8": 9,
    "9": 8
  },
  "d_model": {
    "bidirectional": true,
    "bidirectional_strategy": "add",
    "bidirectional_weight_tie": true,
    "complement_map": {
      "0": 11718,
      "1": 8593,
      "10": 10468,
      "100": 11843,
      "1000": 11638,
      "10000": 11710,
      "10001": 8585,
      "10002": 5460,
      "10003": 2335,
      "10004": 14835,
      "10005": 11085,
      "10006": 7960,
      "10007": 4835,
      "10008": 1710,
      "10009": 14210,
      "1001": 8513,
      "10010": 10460,
      "10011": 7335,
      "10012": 4210,
      "10013": 1085,
      "10014": 13585,
      "10015": 9835,
      "10016": 6710,
      "10017": 3585,
      "10018": 46

In [9]:
caduceus_cfg.d_model

{'_target_': 'caduceus.configuration_caduceus.CaduceusConfig',
 'd_model': 256,
 'n_layer': 16,
 'vocab_size': 15625,
 'ssm_cfg': {'d_state': 16,
  'd_conv': 4,
  'expand': 2,
  'dt_rank': 'auto',
  'dt_min': 0.001,
  'dt_max': 0.1,
  'dt_init': 'random',
  'dt_scale': 1.0,
  'dt_init_floor': 0.0001,
  'conv_bias': True,
  'bias': False,
  'use_fast_path': True},
 'rms_norm': True,
 'fused_add_norm': True,
 'residual_in_fp32': False,
 'pad_vocab_size_multiple': 1,
 'norm_epsilon': 1e-05,
 'initializer_cfg': {'initializer_range': 0.02,
  'rescale_prenorm_residual': True,
  'n_residuals_per_layer': 1},
 'bidirectional': True,
 'bidirectional_strategy': 'add',
 'bidirectional_weight_tie': True,
 'rcps': True,
 'complement_map': {'0': 11718,
  '1': 8593,
  '2': 5468,
  '3': 2343,
  '4': 14843,
  '5': 11093,
  '6': 7968,
  '7': 4843,
  '8': 1718,
  '9': 14218,
  '10': 10468,
  '11': 7343,
  '12': 4218,
  '13': 1093,
  '14': 13593,
  '15': 9843,
  '16': 6718,
  '17': 3593,
  '18': 468,
  '19

In [10]:
caduceus_cfg

CaduceusConfig {
  "bidirectional": true,
  "bidirectional_strategy": "add",
  "bidirectional_weight_tie": true,
  "complement_map": {
    "0": 0,
    "1": 1,
    "10": 7,
    "11": 11,
    "12": 12,
    "13": 13,
    "14": 14,
    "15": 15,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 10,
    "8": 9,
    "9": 8
  },
  "d_model": {
    "_target_": "caduceus.configuration_caduceus.CaduceusConfig",
    "bidirectional": true,
    "bidirectional_strategy": "add",
    "bidirectional_weight_tie": true,
    "complement_map": {
      "0": 11718,
      "1": 8593,
      "10": 10468,
      "100": 11843,
      "1000": 11638,
      "10000": 11710,
      "10001": 8585,
      "10002": 5460,
      "10003": 2335,
      "10004": 14835,
      "10005": 11085,
      "10006": 7960,
      "10007": 4835,
      "10008": 1710,
      "10009": 14210,
      "1001": 8513,
      "10010": 10460,
      "10011": 7335,
      "10012": 4210,
      "10013": 1085,
      "10014": 13585,
      "10015":

In [6]:
from src.models.sequence.dna_embedding import DNAEmbeddingModelCaduceus
backbone = DNAEmbeddingModelCaduceus(config=caduceus_cfg)

TypeError: empty(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got dict"

In [8]:
caduceus_cfg

TypeError: '<' not supported between instances of 'int' and 'str'

In [19]:
#bro I'm so dumb, I'm not unpacking!!!
caduceus_cfg = CaduceusConfig(**cfg['model']['config'])

In [20]:
caduceus_cfg

CaduceusConfig {
  "bidirectional": true,
  "bidirectional_strategy": "add",
  "bidirectional_weight_tie": true,
  "complement_map": {
    "0": 11718,
    "1": 8593,
    "10": 10468,
    "100": 11843,
    "1000": 11638,
    "10000": 11710,
    "10001": 8585,
    "10002": 5460,
    "10003": 2335,
    "10004": 14835,
    "10005": 11085,
    "10006": 7960,
    "10007": 4835,
    "10008": 1710,
    "10009": 14210,
    "1001": 8513,
    "10010": 10460,
    "10011": 7335,
    "10012": 4210,
    "10013": 1085,
    "10014": 13585,
    "10015": 9835,
    "10016": 6710,
    "10017": 3585,
    "10018": 460,
    "10019": 12960,
    "1002": 5388,
    "10020": 12335,
    "10021": 9210,
    "10022": 6085,
    "10023": 2960,
    "10024": 15460,
    "10025": 11585,
    "10026": 8460,
    "10027": 5335,
    "10028": 2210,
    "10029": 14710,
    "1003": 2263,
    "10030": 10960,
    "10031": 7835,
    "10032": 4710,
    "10033": 1585,
    "10034": 14085,
    "10035": 10335,
    "10036": 7210,
    "10037

In [22]:
from src.models.sequence.dna_embedding import DNAEmbeddingModelCaduceus
backbone = DNAEmbeddingModelCaduceus(config=caduceus_cfg)

In [23]:
backbone

DNAEmbeddingModelCaduceus(
  (caduceus): Caduceus(
    (backbone): CaduceusMixerModel(
      (embeddings): CaduceusEmbeddings(
        (word_embeddings): RCPSEmbedding(
          (embedding): Embedding(15625, 256)
        )
      )
      (layers): ModuleList(
        (0-15): 16 x RCPSMambaBlock(
          (mixer): RCPSWrapper(
            (submodule): BiMambaWrapper(
              (mamba_fwd): Mamba(
                (in_proj): Linear(in_features=256, out_features=1024, bias=False)
                (conv1d): Conv1d(512, 512, kernel_size=(4,), stride=(1,), padding=(3,), groups=512)
                (act): SiLU()
                (x_proj): Linear(in_features=512, out_features=48, bias=False)
                (dt_proj): Linear(in_features=16, out_features=512, bias=True)
                (out_proj): Linear(in_features=512, out_features=256, bias=False)
              )
              (mamba_rev): Mamba(
                (in_proj): Linear(in_features=256, out_features=1024, bias=False)
            

In [24]:

backbone.load_state_dict(model_state_dict, strict=True)

NameError: name 'model_state_dict' is not defined

# reset and do it all again

In [36]:
#we have the hyena utils, let's see if we can quickly adapt it for caduceus

import sys
sys.path.append('/data/leslie/sarthak/caduceus')
from src.models.sequence.dna_embedding import DNAEmbeddingModel
from src.tasks.decoders import EnformerDecoder
# from src.tasks.encoders import EnformerEncoder
import torch
import numpy as np
import src.dataloaders.datasets.enformer_dataset as enformer_dataset
import yaml
from omegaconf import OmegaConf
import os
import matplotlib.pyplot as plt
# import seaborn as sns
from tqdm import tqdm
try:
    OmegaConf.register_new_resolver('eval', eval)
    OmegaConf.register_new_resolver('div_up', lambda x, y: (x + y - 1) // y)
except ValueError as e:
    if "Resolver already registered" in str(e):
            print(f"Resolver already exists, skipping registration.")
class IdentityNet(torch.nn.Module):
    def __init__(self):
        super(IdentityNet, self).__init__()

    def forward(self, x):
        return x, None
ckpt_path = '/data/leslie/sarthak/caduceus/outputs/2024-08-06/16-29-46-520749/checkpoints/05-val_loss=0.61190.ckpt'
model_cfg_path = os.path.join(os.path.dirname(os.path.dirname(ckpt_path)), '.hydra', 'config.yaml')
cfg = yaml.load(open(model_cfg_path, 'r'), Loader=yaml.FullLoader)
cfg = OmegaConf.create(cfg)
cfg = OmegaConf.to_container(cfg, resolve=True)
state_dict = torch.load(ckpt_path, map_location='cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_args = cfg['dataset']
split = 'test'
dataset = enformer_dataset.EnformerDataset(split, dataset_args['max_length'], rc_aug = dataset_args['rc_aug'],
                                                            return_CAGE=dataset_args['return_cage'], cell_type=dataset_args.get('cell_type', None),
                                                            kmer_len=dataset_args['kmer_len']) #could use dataloader instead, but again kinda complex

Using kmer genome with length 6


In [37]:
cfg['decoder']['d_output'] = dataset.d_output     
torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
    state_dict["state_dict"], "model."
)
model_state_dict = state_dict["state_dict"]
# need to remove torchmetrics. to remove keys, need to convert to list first
for key in list(model_state_dict.keys()):
    if "torchmetrics" in key:
        model_state_dict.pop(key)
# the state_dict keys slightly mismatch from Lightning..., so we fix it here
decoder_state_dict = {}
for key in list(model_state_dict.keys()):
    if "decoder" in key:
        decoder_state_dict[key[10:]] = model_state_dict.pop(key)
encoder_state_dict = {}
for key in list(model_state_dict.keys()):
    if "encoder" in key:
        encoder_state_dict[key[10:]] = model_state_dict.pop(key)

In [38]:
cfg['model']['config'].pop('_target_')
cfg['model']['config']['complement_map'] = dataset.tokenizer.complement_map
from caduceus.configuration_caduceus import CaduceusConfig
caduceus_cfg = CaduceusConfig(**cfg['model']['config'])

In [39]:
from src.models.sequence.dna_embedding import DNAEmbeddingModelCaduceus
backbone = DNAEmbeddingModelCaduceus(config=caduceus_cfg)

In [40]:
backbone.load_state_dict(model_state_dict, strict=True)

<All keys matched successfully>

In [41]:
#ahh perfect!!
cfg['decoder']

{'_name_': 'enformer',
 'mode': 'pool',
 'd_model': 256,
 'd_output': 4675,
 'kmer_len': 6,
 'conjoin_train': True,
 'conjoin_test': False,
 'convolutions': False}

In [42]:
del cfg['decoder']['_name_']

In [43]:
decoder = EnformerDecoder(**cfg['decoder'])

In [44]:
decoder.load_state_dict(decoder_state_dict, strict=True)

<All keys matched successfully>

In [45]:
encoder_state_dict #nothing cuz uses embedding!

{}

In [46]:
#let's try using the utils
import evals.evals_utils_enformer as e

In [47]:
mamba = e.Evals('/data/leslie/sarthak/caduceus/outputs/2024-08-06/16-29-46-520749/checkpoints/05-val_loss=0.61190.ckpt')

Using kmer genome with length 6


AttributeError: 'NoneType' object has no attribute 'tokenizer'

# opos redo

In [2]:
import sys
sys.path.append('/data/leslie/sarthak/caduceus')
import evals.evals_utils_enformer as e
mamba = e.Evals('/data/leslie/sarthak/caduceus/outputs/2024-08-06/16-29-46-520749/checkpoints/05-val_loss=0.61190.ckpt')

Using kmer genome with length 6


In [3]:
mamba(0)

tensor([[[0.0775, 0.1143, 0.1089,  ..., 0.1833, 0.1910, 0.1354],
         [0.0976, 0.1779, 0.2419,  ..., 0.1137, 0.1177, 0.0743],
         [0.1061, 0.1426, 0.1487,  ..., 0.0868, 0.0896, 0.0658],
         ...,
         [0.1201, 0.1174, 0.1328,  ..., 1.4166, 1.6805, 0.6578],
         [0.1483, 0.1280, 0.1461,  ..., 1.5787, 1.9182, 0.6982],
         [0.1628, 0.1366, 0.1464,  ..., 1.1149, 1.3351, 0.5606]]],
       device='cuda:0')

In [4]:
#eyo it works!!!!
#print the output of the last cell
print(_.shape) #yeah exactly as we'd expect!!

torch.Size([1, 896, 4675])
